20240218-1424: merged to main, split calculate function x,y

In [1]:
##### Library #####
import cv2
import numpy as np
import matplotlib.pyplot as plt

import time
import DobotAPI
import DobotTypes
from DobotControl import DobotControl

In [2]:
##### Function ######

# Main 
def Mask_n_Draw(_hsv, _draw_frame):
    frame = _draw_frame
    # HSV color boundary 0-179
    # HSV boundry list [red_lower, red_upper]
    hsv_red1   = [np.array([  0, 86, 0]), np.array([ 5,255,255])]
    hsv_red2   = [np.array([  172, 68, 0]), np.array([ 180,255,255])]
    hsv_yellow = [np.array([ 27, 6, 186]), np.array([ 33,255,255])]
    hsv_green = [np.array([ 56, 105, 17]), np.array([ 77,206,228])]
    hsv_blue  = [np.array([127, 79, 56]), np.array([127,232,249])]
    hsv_purple  = [np.array([153, 54, 26]), np.array([153,193,174])]

    # create mask image
    mask_red1   = cv2.inRange(frame_hsv, hsv_red1[0], hsv_red1[1])
    mask_red2   = cv2.inRange(frame_hsv, hsv_red2[0], hsv_red2[1])
    mask_yellow = cv2.inRange(frame_hsv, hsv_yellow[0], hsv_yellow[1])
    mask_green = cv2.inRange(frame_hsv, hsv_green[0], hsv_green[1])
    mask_blue  = cv2.inRange(frame_hsv, hsv_blue[0], hsv_blue[1])
    mask_purple  = cv2.inRange(frame_hsv, hsv_purple[0], hsv_purple[1])

    # merge red1 and red2
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)
    
    # Contour Retrieval Mode
    # https://docs.opencv.org/3.4/d9/d8b/tutorial_py_contours_hierarchy.html
    CV2_retr = cv2.RETR_TREE # cv2.RETR_TREE, cv2.RETR_LIST, cv2.RETR_EXTERNAL
    # Contour Approximation Method
    # https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html
    CV2_approx = cv2.CHAIN_APPROX_SIMPLE # cv2.CHAIN_APPROX_NONE, cv2.CHAIN_APPROX_SIMPLE

    contours_red, _ = cv2.findContours(mask_red, CV2_retr, CV2_approx)
    contours_yellow, _ = cv2.findContours(mask_yellow, CV2_retr, CV2_approx)
    contours_green, _ = cv2.findContours(mask_green, CV2_retr, CV2_approx)
    contours_blue, _ = cv2.findContours(mask_blue, CV2_retr, CV2_approx)
    contours_purple, _ = cv2.findContours(mask_purple, CV2_retr, CV2_approx)

    Contour_draw_Rbox(contours_red, 500, (0,0,255), 'red', frame)
    Contour_draw_Rbox(contours_yellow, 500, (0,255,255), 'yellow', frame)
    Contour_draw_Rbox(contours_green, 500, (0,255,0), 'green', frame)
    Contour_draw_Rbox(contours_blue, 500, (255,0,0), 'blue', frame)
    Contour_draw_Rbox(contours_purple, 500, (255,0,255), 'purple', frame)
    Contour_draw_Claw(contours_red, 500, (0,0,255), 'red', frame, 0)
    Contour_draw_Claw(contours_yellow, 500, (0,255,255), 'yellow', frame, 1)
    Contour_draw_Claw(contours_green, 500, (0,255,0), 'green', frame, 2)
    Contour_draw_Claw(contours_blue, 500, (255,0,0), 'blue', frame, 3)
    Contour_draw_Claw(contours_purple, 500, (255,0,255), 'purple', frame, 4)

    return frame


# normal box
def Contour_draw_box(Contours, min_area, box_color, text, frame2draw):
    for cnt in Contours:
        if cv2.contourArea(cnt) > min_area:
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(frame2draw, (x,y), (x+w,y+h), box_color, 2)
            if text != '':
                cv2.putText(frame2draw, text, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, box_color, 2)

# rotate box
def Contour_draw_Rbox(Contours, min_area, box_color, text, frame2draw):
    for cnt in Contours:
        if cv2.contourArea(cnt) > min_area:
            x,y,w,h = cv2.boundingRect(cnt)
            rect = cv2.minAreaRect(cnt)
            rotate = rect[2]
            box = cv2.boxPoints(rect)
            box = np.int32(box)

            cv2.drawContours(frame2draw, [box], 0, box_color, 1)
            if text != '':
                cv2.putText(frame2draw, text+f': {rotate:.1f}', (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, box_color, 2)

# draw claw line
def Contour_draw_Claw(Contours, min_area, box_color, text, frame2draw, object_list_number):
    for cnt in Contours:
        if cv2.contourArea(cnt) > min_area:
            x,y,w,h = cv2.boundingRect(cnt)
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.int32(box)
            rotate = rect[2]
            xy_cen = np.int32(rect[0])

            wing_12 = (int((box[1][0]+box[2][0])/2),int((box[1][1]+box[2][1])/2))
            wing_03 = (int((box[0][0]+box[3][0])/2),int((box[0][1]+box[3][1])/2))
            cv2.circle(frame2draw, xy_cen, 0, box_color, 10)
            cv2.arrowedLine(frame2draw, xy_cen, wing_12, box_color, 1)
            cv2.arrowedLine(frame2draw, xy_cen, wing_03, box_color, 1)
            if text != '':
                cv2.putText(frame2draw, text+f': {rotate:.1f}', (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, box_color, 2)
            
            global object_list_status
            global object_list
            temp_list = []

            if object_list_status == True:
                object_list[object_list_number].append(((xy_cen[0],xy_cen[1]), rotate))


In [3]:
# Initiate Dobot
class Robot(DobotControl):
    def __init__(self):
        super().__init__()

if __name__ == '__main__':
    dobot = Robot()
    dobot.setAddr(Robot.search()[0])
    dobot.startRobot()

class DC: # Dobot Camera offset to 0-100 percent
    def __init__(self): # target cam pixel
        self.dobot_min_x = 214.10 # 'dobot west most'
        self.dobot_max_x = 368.69 # 'dobot east most'
        self.dobot_min_y = -97.06 # 'dobot south most'
        self.dobot_max_y = 123.17 #'dobot north most'
        self.dobot_min_z = 100 # 'dobot bottom most'
        self.dobot_max_z = 150 # 'dobot upper most'
        self.cam_min_x =  92 # 'cam west most'
        self.cam_max_x = 511 # 'cam east most'
        self.cam_min_y =  61 # 'cam north most'
        self.cam_max_y = 350 # 'cam south most'
        self.dobot_dif_x = self.dobot_max_x - self.dobot_min_x # WorkSpace size
        self.dobot_dif_y = self.dobot_max_y - self.dobot_min_y
        self.cam_dif_x = self.cam_max_x - self.cam_min_x
        self.cam_dif_y = self.cam_max_y - self.cam_min_y
        # !!! opencv cam flip Y axis (pixel 0 at top)
        # webcam -> dobot --> rotate 180 degree

    def cam2botX(self,camX):
        camX = (self.cam_dif_x - (camX - self.cam_min_x)) / self.cam_dif_x
        botX = (camX * self.dobot_dif_x) + self.dobot_min_x
        return  botX
    
    def cam2botY(self,camY):
        camY = (self.cam_dif_y - (camY - self.cam_min_y)) / self.cam_dif_y 
        camY = 1 - camY # opencv cam flip Y
        botY = (camY * self.dobot_dif_y) + self.dobot_min_y
        return  botY

# Homing Dobot to close to base
dobot_test = DC()
dobot.moveTo(155, -12.6, 100, 40) # Absolute Position

['COM3']
COM3 Connect status: DobotConnect_Successfully
GetDeviceVersion: V2.1.1
Initing dobot COM3
Stopped queue successfully


Device version may be not supported: 2.1.1 Dobot[COM3]


COM3 move to 155.0 -12.6 100.0 40.0


In [4]:
##### Main 1 - Realtime Viewing ######
print('>> Phase 1')

object_list_status = False # for list process later

# Read Video/ Cam
# cap = cv2.VideoCapture('UpskillReskill_OpenCV_2023/shibuya.mp4')
cap = cv2.VideoCapture(1)

vid_w = int(cap.get(3))
vid_h = int(cap.get(4))
vid_fps = int(cap.get(cv2.CAP_PROP_FPS))
vid_scale = 1
print(f'video size = {vid_w} x {vid_h}\nframerate = {vid_fps}')

# fix distortion
dist = np.array([[-0.189400, -0.027179, -0.003365, 0.001168, 0.0]])
cameraMatrix = np.array([[878.507989, 0.        , 331.792216],
                         [0.        , 876.710299, 235.020060],
                         [0.        , 0.        , 1.]])
newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, dist, (vid_w,vid_h), 1, (vid_w,vid_h))

while (cap.isOpened()):
    # read and check for image availability before continue
    check, frame = cap.read()
    if check != True:
        print('>> video ended or not available')
        break

    # pre-process image
    
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    frame = cv2.resize(frame, (int(vid_w*vid_scale),int(vid_h*vid_scale)))
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # backup for last frame from video
    frame_lastClean = cv2.resize(frame, (int(vid_w*vid_scale),int(vid_h*vid_scale)))

    # start drawing
    frame_drawed = Mask_n_Draw(frame_hsv, frame)

    # show window
    cv2.imshow('RGB', frame_drawed)
    # cv2.imshow('R', mask_red)
    # cv2.imshow('G', mask_green)
    # cv2.imshow('B', mask_blue)

    if cv2.waitKey(1) & 0xFF == ord('e'):
        print('>> Keyboard Exit')
        break
cap.release()
cv2.destroyAllWindows()

# Show last captured frame
# cv2.imshow('RGB_clean', frame_lastClean) # last frame without drawing
# cv2.imshow('RGB_draw', frame_drawed) # last frame with doodle
# cv2.waitKey(0)
cv2.destroyAllWindows()

>> Phase 1
video size = 640 x 480
framerate = 30
>> Keyboard Exit


In [5]:
##### Main 2 - Still Image Process ######
print('>> Phase 2')

object_list_status = True
object_list = [[],[],[],[],[]] # R,Y,G,B,P

frame = frame_lastClean
frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

frame_drawed = Mask_n_Draw(frame_hsv, frame)

# amount of object found
obj_red = len(object_list[0])
obj_yellow = len(object_list[1])
obj_green = len(object_list[2])
obj_blue = len(object_list[3])
obj_purple = len(object_list[4])
print(f'Object count\n   red   : {obj_red:>2}\n   yellow : {obj_yellow:>2}\n   green  : {obj_green:>2}\n   blue  : {obj_blue:>2}\n   purple  : {obj_purple:>2}')

print(object_list[0]) # show red location
print(object_list[1]) # Y
print(object_list[2]) # G
print(object_list[3]) # B
print(object_list[4]) # P

cv2.imshow('RGB_Still', frame_drawed)
cv2.waitKey(0)
cv2.destroyAllWindows()

>> Phase 2
Object count
   red   :  2
   yellow :  0
   green  :  2
   blue  :  0
   purple  :  0
[((507, 350), 90.0), ((98, 63), 90.0)]
[]
[((90, 350), 88.31531524658203), ((510, 64), 90.0)]
[]
[]


In [6]:

### Write Your Code Here ######################################################
# dobot.moveTo(x, y, z, r) # Absolute Position
# dobot.moveInc(dx, dy, dz, dr) # Relative Position
# dobot.setPump(17, 18) # Active Pump
# dobot.resetPump(17, 18) # Deactive Pump
# dobot.suck() # S U C K
# dobot.unsuck # un S U C K
# time.sleep(1) # sleep 1 sec

aaaa = DC()
# object_list [color blog][number in list][0=xy 1=r][0=x 1=y only in xy]
print('x=',object_list[0][0][0][0], ' y=',object_list[0][0][0][1])
# print(object_list) # everything
# print(object_list[0]) # all red
# print(object_list[0][0]) # red 1st
# print(object_list[0][0][0]) # xy in red first
# print(object_list[0][0][0][0]) # just x in red first
x = aaaa.cam2botX(object_list[0][0][0][0])
y = aaaa.cam2botY(object_list[0][0][0][1])
# theta = aaaa.cam2botY(object_list[0][0][1])
dobot.moveTo(x, y, 70, 40) # Absolute Position
time.sleep(3) # sleep 1 sec

x = aaaa.cam2botX(object_list[0][1][0][0])
y = aaaa.cam2botY(object_list[0][1][0][1])
dobot.moveTo(x, y, 70, 40) # Absolute Position
time.sleep(3) # sleep 1 sec

x = aaaa.cam2botX(object_list[2][0][0][0])
y = aaaa.cam2botY(object_list[2][0][0][1])
dobot.moveTo(x, y, 70, 40) # Absolute Position
time.sleep(3) # sleep 1 sec

x = aaaa.cam2botX(object_list[2][1][0][0])
y = aaaa.cam2botY(object_list[2][1][0][1])
dobot.moveTo(x, y, 70, 40) # Absolute Position
time.sleep(3) # sleep 1 sec

x= 507  y= 350
COM3 move to 215.57579952267304 123.17000000000002 70.0 40.0
COM3 move to 366.4763007159904 -95.5359169550173 70.0 40.0
COM3 move to 369.42789976133656 123.17000000000002 70.0 40.0
COM3 move to 214.46894988066825 -94.77387543252594 70.0 40.0
